## Logistic Regression

Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

In [15]:
bcancer = pd.read_csv("files/breast-cancer-wisconsin.csv", na_values="?")
bcancer.head()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [16]:
bcancer.shape

(699, 11)

In [13]:
# Examine the data, look at the statistical summary etc.
bcancer.describe()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,683.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.544656,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,3.643857,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [19]:
# deal with the NaN values in the data
bcancer.isna().sum()

sample_code_number              0
clump_thickness                 0
uniformity_cell_size            0
uniformity_cell_shape           0
marginal_adhesion               0
single_epithelial_cell_size     0
bare_nuclei                    16
bland_chromatin                 0
normal_nucleoli                 0
mitoses                         0
class                           0
dtype: int64

In [23]:
clean_bcancer = bcancer.dropna()
clean_bcancer.shape

(683, 11)

In [30]:
# Build your first model - defining training and test data sets then use Logistic Regression to build a model
train, test= train_test_split(clean_bcancer, test_size = 0.2, random_state = 142)
print("train shape: ", train.shape)
print("test shape: ", test.shape)

train shape:  (546, 11)
test shape:  (137, 11)


In [34]:
X_train = train.drop(['sample_code_number', 'class'], axis=1)
y_train = train['class']
X_test = test.drop(['sample_code_number', 'class'], axis=1)
y_test = test['class']
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (546, 9)
y_train shape:  (546,)
X_test shape:  (137, 9)
y_test shape:  (137,)


In [38]:
model = LogisticRegression()
model.fit(X_train, y_train)

C:\Users\b0kch\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [41]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred

array([2, 4, 2, 2, 4, 4, 2, 4, 2, 4, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 4, 4, 4, 2, 2, 4, 2, 2, 2, 4, 2, 4,
       2, 4, 2, 4, 2, 2, 2, 4, 2, 2, 4, 2, 4, 4, 4, 2, 2, 4, 2, 2, 4, 2,
       4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 4, 2, 2, 4,
       2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 4, 4, 2, 2, 2, 2, 2, 4, 2,
       2, 4, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 4, 4, 2, 2, 4, 2, 4, 2, 2, 2,
       2, 4, 2, 4, 4], dtype=int64)

In [43]:
y_test

280    2
232    2
369    2
563    2
491    4
320    4
327    2
270    4
63     4
187    4
50     4
632    2
45     2
304    4
573    2
561    2
679    2
135    2
13     2
384    2
169    2
527    2
167    4
198    2
671    2
502    2
255    4
538    2
470    2
105    4
      ..
589    2
523    4
143    2
602    2
425    4
609    2
218    4
72     2
281    2
279    4
124    4
672    2
510    2
347    2
4      2
247    4
122    4
354    2
667    2
177    4
365    2
233    4
615    2
419    2
432    2
645    2
353    4
307    2
126    4
67     4
Name: class, Length: 137, dtype: int64

In [46]:
# calculating test accuracy
print("Accuracy on test set")
print(accuracy_score(y_pred, y_test))

Accuracy on test set
0.9635036496350365


In [49]:
# printing confusion matrix
confusion_matrix(y_test, y_pred)

array([[83,  2],
       [ 3, 49]], dtype=int64)

### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

In [75]:
model = LogisticRegression()
selector = RFE(model, 4, step=1)
selector.fit(X_train, y_train)
y_pred = selector.predict(X_test)
print("Accuracy on test set with 4 features :", accuracy_score(y_test, y_pred))
print(selector.support_)
column_Names = X_train.columns
print(column_Names[selector.support_])

Accuracy on test set with 5 features : 0.9635036496350365
[ True  True False False False  True  True False False]
Index(['clump_thickness', 'uniformity_cell_size', 'bare_nuclei',
       'bland_chromatin'],
      dtype='object')


C:\Users\b0kch\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\b0kch\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\b0kch\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\b0kch\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\b0kch\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

## Conclusion

The logistic regression model developed for the classification of breast cancer tumors demonstrates a robust predictive capability with an accuracy of approximately 96.35% on the test set. The confusion matrix reveals that the model is more likely to predict false negatives than false positives. In a clinical diagnostic setting, the implications of these two types of errors are vastly different. False positives, while less common in this model, could lead to unnecessary anxiety and additional testing for patients, but false negatives are more concerning as they may result in delayed diagnosis and treatment for patients with malignant tumors.

Through the process of feature selection using Recursive Feature Elimination (RFE), it was observed that a model with only four features could achieve the same level of accuracy as models with more features. This suggests that a simpler model with fewer features can be just as effective while also being more computationally efficient, which is beneficial for scaling the application to larger datasets or implementing it in real-time diagnostic tools.

The most significant features contributing to the model's predictions were identified, with clump thickness being particularly influential. This insight aligns with medical understanding that certain cellular characteristics are indicative of malignancy.

In conclusion, the logistic regression model serves as a potentially valuable tool for assisting in the diagnosis of breast cancer. It provides high accuracy with a manageable number of features, making it suitable for clinical applications. However, it is crucial to minimize false negatives further to ensure that patients receive timely and appropriate care. Future work should focus on refining the model to improve its sensitivity to malignant cases without significantly increasing the false positive rate.